# Semantic Search

In [ ]:
from qdrant_client import models, QdrantClient 
from sentence_transformers import SentenceTransformer

## Langchain e Qdrant

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings

/home/miguel/miniforge3/envs/DataScience/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
matryoshka_dim = 1024

embedd_model_name="Qwen/Qwen3-Embedding-0.6B"

model_kwargs = {"device": "cpu", 'truncate_dim':matryoshka_dim}
encode_kwargs = {"normalize_embeddings": False} # "prompt_name": 'query'}

# Conecta com LangChain
embeddings = HuggingFaceEmbeddings(model_name=embedd_model_name, 
                                   model_kwargs=model_kwargs,
                                   encode_kwargs=encode_kwargs)

In [ ]:
#ids_doc = vector_store_doc.add_documents(documents=documents)

'''
Transforma Documents em vetores e os armazena em uma vector store
9990 documents demorou 33min


Notas:

    Falta colocar os metadados em Documents class para aprimorar a busca
'''

#### Salva vector store em disco

In [ ]:
'''

Salva vector store em documento

'''
#vector_store_doc.save_local("9990 catalogos") 

#### Carrega vector store de disco

In [ ]:
#vector_store_doc.load_local("9990 catalogos", embeddings,  allow_dangerous_deserialization=True) # Carrega vector store de documento

#### Testes

In [ ]:
documents

In [ ]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery:{query}'

# Each query must come with a one-sentence instruction that describes the task
tasks = [['Dada uma consulta de pesquisa na web, recuperar trechos relevantes que respondam à consulta', "Given a web search query, retrieve relevant passages that answer the query"],
        ['Dada uma consulta de pesquisa de catálogo, recuperar passagens relevantes que respondam à consulta', 'Given a catalog search query, retrieve relevant passages that answer the query']]

task = tasks[0][0]

queries = [
    get_detailed_instruct(task, '"Gostaria de obter dados do bolsa familia de 2025"'),
    get_detailed_instruct(task, "Quero saber onde a há mais crimes no brasil"),
    get_detailed_instruct(task, "quero dados sobre internação hospitalar no hospita julio muler"),
    get_detailed_instruct(task, "quero saber onde mais chove no Rio de Janeiro")
]

In [ ]:
query = queries[0]

results = vector_store_doc.similarity_search_with_score(
    query, k=20)

# Returns: List of documents most similar to the query text with L2 distance in float. Lower score represents more similarity

print(query)
print()

for k in results:

    print(k[0])
    print(k[1])
    print()

'''
Notas:

    Alguns resultados performam melhor que a busca padrão do GOV, 
    lidando maior com ambiguidades e sinonimos. Entretanto em outros resultados
    passa longe do esperado e em geral algumas respotas são um pouco fora de 
    contexto. 

    Necessário buscar um enriquecimento de como os dados foram representados
    na fase de embedding como metadados, ids, descrições de cada catalogo, 
    adicionar metadados como ano, local.., tambem pode ser necessario um
    modelo maior ou ajustar tamanho da dimensão dos vetores.

    Outras tecnicas como matryoshka e podem ajudar a aumentar a acurácia
    das buscas
    https://huggingface.co/blog/matryoshka

    Além disso temos full-text search e ReRanking que pode ajudar no desempenho também
    
    É interessante estudar sobre asymmetric semantic search, pois a query e 
    a base de dados podem ter tamanho bem diferentes ler em: 
    https://www.sbert.net/examples/sentence_transformer/applications/semantic-search/README.html

'''

### Extração de metadados

**Estrutura o metadado:** ['mes': str'(1-12)', 'ano': str, ]

#### sentenceTransformers

In [ ]:
meses = ['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho',
         'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

calendario = []

for ano in range(2000, 2030):
    for mes in meses:
        calendario.append(f'{mes} de {ano}')

calendario

In [ ]:
vector1 = embeddings.embed_query("Quero dados de crimes em 2017")
vector2 = embeddings.embed_documents(calendario)

# assert verifica a condição caso não seja verdadeira, retorna um erro
assert len(vector1) == len(vector2[0]) # verifica se ambos vetores possuem a mesma dimensão, caso contrário, retorna um erro

print(len(vector1))
print(vector1[:10])

print(len(vector2))
print(vector2[:10])

In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot = np.dot(vec1, vec2)
    return dot / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

query = embeddings.embed_query("Quero dados de crimes em 2017")
doc = embeddings.embed_documents(calendario)


In [ ]:
sim = []

for vec in doc:
    similarity = cosine_similarity(query, vec)
    sim.append(similarity)
    print("Cosine Similarity:", similarity)

'''
Permite verificar similaridade entre vetores
permite armazenar vetores temporiariamente em cache para evitar recomputar

https://docs.langchain.com/oss/python/integrations/text_embedding/index#huggingface

'''


#### faiss e LangChain

In [ ]:
meses = ['', 'janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho',
         'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

calendario = []

for ano in range(2000, 2030):
    for mes in range(13):
        calendario.append(f'{mes}/{ano}')

calendario

In [ ]:
embedding_dim = len(embeddings.embed_query("Data de ano tal")) # dimensão do vetor de embedding
index = faiss.IndexFlatL2(embedding_dim) # L2 é a distância euclidiana

vector_store_met = FAISS(embedding_function=embeddings, 
                     index=index, 
                     docstore=InMemoryDocstore({}), 
                     index_to_docstore_id={}) # cria objeto vectorstore

In [ ]:
from langchain_core.documents import Document

In [ ]:
meta = []
for k in calendario:

    meta.append(Document(page_content=k, metadata={}))

In [ ]:
ids_met = vector_store_met.add_documents(documents=meta)

In [ ]:
result = vector_store_met.similarity_search_with_score(
    "2017", k=20)

In [ ]:
print(result)